In [ ]:
!pip install gradio
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
from google.colab import userdata
OpenAIkey = userdata.get('OpenAI')
SegmindAPIKey = userdata.get('Segmind')

print (len(OpenAIkey))
print (len(SegmindAPIKey))

132
19


In [ ]:
import gradio as gr
import requests
import base64
import random
import io
from PIL import Image

# Define your functions here
def getbase64image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def getImageCaption(b64image):
    chatgpt_payload = {
    "model":"gpt-4-vision-preview",
    "messages":[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "Create a concise yet fully descriptive one sentence caption for the image to use on a Stock Photo Website"},
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{b64image}",
            },
          },
        ],
      }
    ],
    "max_tokens":300,
    }

    # Make the request to OpenAI's API
    response = requests.post(chatgpt_url, json=chatgpt_payload, headers=chatgpt_headers)
    response_json = response.json()
    caption = response_json['choices'][0]['message']['content']
    return caption


def generate_images(base64image, imagecaption, count):
    SegmindAPIKey = userdata.get('Segmind')
    url = "https://api.segmind.com/v1/ssd-img2img"
    generated_image_filenames = []

    for i in range(count):
        currentseed = random.randint(1000, 1000000)

        # Request payload
        data = {
            "image": base64image,
            "prompt": imagecaption + ", stock photo",
            "negative_prompt": "ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, watermark, grainy, signature, cut off, draft",
            "samples": 1,
            "scheduler": "UniPC",
            "num_inference_steps": 30,
            "guidance_scale": "7.5",
            "seed": currentseed,
            "strength": "0.9",
            "base64": False
        }

        response = requests.post(url, json=data, headers={'x-api-key': SegmindAPIKey})

        if response.status_code == 200 and response.headers.get('content-type') == 'image/jpeg':
            image_data = response.content
            image = Image.open(io.BytesIO(image_data))
            image_filename = f"image{i + 1}.jpg"
            image.save(image_filename)
            generated_image_filenames.append(image_filename)

    return generated_image_filenames

def process_image(uploaded_image, count):
    base64image = getbase64image(uploaded_image)
    #imagecaption = getImageCaption(base64image).strip('\"')
    imagecaption = ''
    generated_images = generate_images(base64image, imagecaption, count)
    return imagecaption, generated_images


# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("## 1-Click Similar Image Generator using AI")
    gr.Markdown("Upload an image to generate similar images using AI.")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil")
            count_dropdown = gr.Dropdown(label="Number of Images", choices=[1, 2, 3, 4, 5],value=3)
            btn_genererate_images = gr.Button('Generate Images')
        with gr.Column():
            gallery = gr.Gallery(label="images", columns=3,preview=True,allow_preview=True,show_download_button=True)
            textbox = gr.Textbox(label = "AI Generated Caption")

    btn_genererate_images.click(fn=process_image, inputs=[image_input, count_dropdown], outputs=[textbox,gallery])

# Run the app
app.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://096b4e5ac6886492da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
